In [6]:
import vendors2.kibot.metadata.load.kibot_metadata as kmd

lfc = kmd.KibotHardcodedContractLifetimeComputer(365, 7)

lfc.compute_lifetime("CLJ17")

ContractLifetime(start_date=Timestamp('2016-03-22 00:00:00'), end_date=Timestamp('2017-03-15 00:00:00'))

In [7]:
lfc = kmd.KibotTradingActivityContractLifetimeComputer()

lfc.compute_lifetime("CLJ17")

ContractLifetime(start_date=Timestamp('2011-11-18 00:00:00'), end_date=Timestamp('2017-03-21 00:00:00'))

In [ ]:
## 

In [11]:
symbols = ["ES", "CL"]
file = "./contracts.csv"

lfc = kmd.KibotHardcodedContractLifetimeComputer(365, 7)

kmd = kmd.ContractsLoader(symbols, file, lfc, refresh=True)

AssertionError: 
################################################################################
* Failed assertion *
'5750'
==
'5749'
################################################################################
